In [ ]:
!pip install prophet pandas numpy scikit-learn plotly streamlit pyomo scipy


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [24]:
!pip install --upgrade nbformat

Defaulting to user installation because normal site-packages is not writeable


In [25]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error
import plotly.express as px
import streamlit as st
from scipy.optimize import minimize  # For Simulated Annealing (via basinhopping, which uses SA internally)
import matplotlib.pyplot as plt

In [26]:
train = pd.read_csv('archive/train.csv', parse_dates=['Date'], low_memory=False)
store = pd.read_csv('archive/store.csv')

In [27]:
data = pd.merge(train, store, on='Store')

In [28]:
# Basic EDA (build on yours): Filter open stores, handle zeros, add features
data = data[data['Open'] == 1]  # Only consider open days
data['Sales'] = data['Sales'].clip(lower=0)  # No negative sales
data['LogSales'] = np.log1p(data['Sales'])  # For stability in forecasting

In [29]:
data['Promo'] = data['Promo'].astype(float)
data['SchoolHoliday'] = data['SchoolHoliday'].astype(float)
data['StateHoliday'] = data['StateHoliday'].map({'0': 0, 'a': 1, 'b': 2, 'c': 3}).fillna(0)  # Encode holidays

In [30]:
data.sort_values('Date', inplace=True)

In [31]:
split_date = data['Date'].max() - pd.Timedelta(weeks=6)
train_data = data[data['Date'] < split_date]
test_data = data[data['Date'] >= split_date]
print(train_data.head())

         Store  DayOfWeek       Date  Sales  Customers  Open  Promo  \
1017190   1097          2 2013-01-01   5961       1405     1    0.0   
1016179     85          2 2013-01-01   4220        619     1    0.0   
1016353    259          2 2013-01-01   6851       1444     1    0.0   
1016356    262          2 2013-01-01  17267       2875     1    0.0   
1016368    274          2 2013-01-01   3102        729     1    0.0   

         StateHoliday  SchoolHoliday StoreType Assortment  \
1017190             1            1.0         b          b   
1016179             1            1.0         b          a   
1016353             1            1.0         b          b   
1016356             1            1.0         b          a   
1016368             1            1.0         b          b   

         CompetitionDistance  CompetitionOpenSinceMonth  \
1017190                720.0                        3.0   
1016179               1870.0                       10.0   
1016353                210.0 

In [37]:
fig = px.line(train_data, x='Date', y='Sales', color='StoreType')
fig.write_html("rossmann_sales_plot.html", auto_open=True)

In [38]:
import numpy as np
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error
import plotly.express as px
import plotly.graph_objects as go
from prophet.plot import plot_components

In [40]:
# Group by StoreType, take the first Store ID in each group, and convert to a dictionary
sample_store_map = train_data.groupby('StoreType')['Store'].first().to_dict()

# Extract the Store IDs to a list for iteration later
sample_stores = list(sample_store_map.values()) 

print(f"Sample stores by type: {sample_store_map}")

mapes = {}
forecasts = {}
models = {}

Sample stores by type: {'a': 530, 'b': 1097, 'c': 365, 'd': 369}


In [41]:
for store_id in sample_stores:
    print(f"\nForecasting for Store {store_id}...")
    
    # Prep data for this store (as before)
    store_train = train_data[train_data['Store'] == store_id][['Date', 'Sales', 'Promo', 'SchoolHoliday', 'StateHoliday']].copy()
    store_train['ds'] = store_train['Date']
    store_train['y'] = np.log1p(store_train['Sales'])  # Log for stability
    
    # Test set
    store_test = test_data[test_data['Store'] == store_id][['Date', 'Sales']].copy()
    store_test['ds'] = store_test['Date']
    store_test['y_true'] = store_test['Sales']
    
    if len(store_train) < 2 * 365:  # Skip if too few days
        print(f"Skipping Store {store_id}: Insufficient data")
        continue


Forecasting for Store 530...

Forecasting for Store 1097...

Forecasting for Store 365...
Skipping Store 365: Insufficient data

Forecasting for Store 369...


In [43]:

# Prophet setup with tuning for Rossmann seasonality
model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,  # Start conservative; increase to 0.1 if underfitting
        seasonality_prior_scale=10.0,  # Allows stronger weekly fits
        holidays_prior_scale=10.0
    )
    
# Add regressors (forward-fill any NaNs)
store_train['Promo'] = store_train['Promo'].fillna(0)
store_train['SchoolHoliday'] = store_train['SchoolHoliday'].fillna(0)
store_train['StateHoliday'] = store_train['StateHoliday'].fillna(0)
model.add_regressor('Promo')
model.add_regressor('SchoolHoliday')
model.add_regressor('StateHoliday')
    
    # Fit
model.fit(store_train[['ds', 'y', 'Promo', 'SchoolHoliday', 'StateHoliday']])
models[store_id] = model

17:22:20 - cmdstanpy - INFO - Chain [1] start processing
17:22:22 - cmdstanpy - INFO - Chain [1] done processing


In [46]:
# Future dataframe: Extend to test period + regressors
future_dates = pd.date_range(start=store_train['ds'].min(), end=test_data['Date'].max(), freq='D')
future = model.make_future_dataframe(periods=0, freq='D')
future = future[future['ds'].isin(future_dates)]
    
# Merge regressors (forward-fill for future)
future_reg = store_train[['ds', 'Promo', 'SchoolHoliday', 'StateHoliday']].copy()

# --- FIX START ---
# 1. Set 'ds' as the index
future_reg = future_reg.set_index('ds')

# 2. Get the last row (with datetime index) and reindex using future_dates
#    We use .loc to ensure we select based on the 'ds' index, which is now a date.
last_row = future_reg.iloc[-1].to_frame().T
extrapolated_reg = last_row.reindex(future_dates, method='ffill')
extrapolated_reg.index.name = 'ds' # Name the index back to 'ds' for the merge

# 3. Concatenate the original and the extrapolated data
future_reg_combined = pd.concat([future_reg, extrapolated_reg])
future_reg_combined = future_reg_combined[~future_reg_combined.index.duplicated(keep='first')] # Remove duplicate dates

# 4. Merge back to 'future' dataframe using 'ds' index
future = future.merge(future_reg_combined.reset_index(), on='ds', how='left').fillna(method='ffill').fillna(0)
# --- FIX END ---
    
# Predict
forecast = model.predict(future)
forecast['yhat_sales'] = np.expm1(forecast['yhat']) # Denormalize

/var/folders/lh/7d8p88ps5rx0l3scbz88rbph0000gn/T/ipykernel_96501/332043583.py:24: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [47]:
# Merge with test for eval
test_forecast = forecast[forecast['ds'].isin(store_test['ds'])][['ds', 'yhat_sales']]
merged_test = store_test.merge(test_forecast, left_on='ds', right_on='ds', how='left')
merged_test['yhat_sales'] = merged_test['yhat_sales'].fillna(merged_test['y_true'] * 0.95)  # Fallback for edge cases
    
mape = mean_absolute_percentage_error(merged_test['y_true'], merged_test['yhat_sales']) * 100
mapes[store_id] = mape
forecasts[store_id] = forecast
print(f"Store {store_id} MAPE: {mape:.2f}%")

# Average MAPE
avg_mape = np.mean(list(mapes.values()))
print(f"\nAverage MAPE across sample stores: {avg_mape:.2f}%")

# Plot: Overlay your EDA-style aggregate with per-store forecasts
fig = go.Figure()
for store_id in sample_stores:
    if store_id in forecasts:
        store_fc = forecasts[store_id]
        fig.add_trace(go.Scatter(x=store_fc['ds'], y=store_fc['yhat_sales'], 
                                 mode='lines', name=f'Store {store_id} Forecast',
                                 line=dict(dash='dash')))

Store 369 MAPE: 5.00%

Average MAPE across sample stores: 5.00%


In [48]:
# Quick aggregate for opt
next_week_demands = []
for store_id in sample_stores:
    if store_id in forecasts:
        fc = forecasts[store_id]
        next_week = fc['yhat_sales'].tail(7).sum()  # Assuming daily, sum last 7
        next_week_demands.append(next_week)
print("Predicted weekly demands:", next_week_demands)
# Plug into cost_function and basinhopping as before

Predicted weekly demands: [np.float64(49564.72086652209)]
